# Übung 2 - Polygonzug



In [83]:
# Libraries
import numpy as np
import pandas as pd

In [92]:
#import data
data_Noise = pd.read_csv("./data/data_Noise.txt", delimiter=";")
data_NoNoise = pd.read_csv("./data/data_NoNoise.txt", delimiter=";")
fixpunkte = pd.read_csv("./data/fixpunkte.txt", delimiter=";")
i_meteo = pd.read_csv("./data/i_meteo.txt", delimiter=";")

# const for calcs
T = 12 #°C
P = 1013.25 #hPa
R = 6371000 #m

# "global" funcs
def radian(gons): return (gons * 2 * np.pi) / 400
def gon(rads): return (rads * 400) / 2 * np.pi

data_NoNoise


,Visur_von,Visur_nach,RI,RII,ZI,ZII,D
0,A,P,165.5718,365.5718,103.4797,296.5203,NaN
1,A,PP1,365.7370,165.7370,96.7342,303.2658,27.4199
2,PP1,A,261.8157,61.8157,103.2658,296.7342,27.4199
3,PP1,PP2,60.9770,260.9770,97.2136,302.7864,25.4368
4,PP2,PP1,70.4558,270.4558,102.7864,297.2136,25.4368
5,PP2,PP3,251.6220,51.6220,98.3621,301.6379,33.4681
6,PP3,PP2,371.0396,171.0396,101.6379,298.3621,33.4681
7,PP3,PP4,151.6620,351.6620,99.3118,300.6882,23.3132
8,PP4,PP3,340.9986,140.9986,100.6882,299.3118,23.3132
9,PP4,E,168.9636,368.9636,100.4986,299.5014,44.8211


In [94]:
# 1 Messdatenaufbereitung
def az_add_or_subtract(az): # adds or subtracts 200 gon to azimut
    if az <= 200:
        return az + 200
    else: return az - 200

## Azimuth and zenith angles average
RI = data_Noise["RI"]
RII = data_Noise["RII"].apply(az_add_or_subtract) # Messung 2. Lage 

ZI = data_Noise["ZI"]
ZII = data_Noise["ZII"] # Messung 2. Lage

# Sanity check (should be small, at max 0.1 otherwise somebody f*cked up)
#print(RI - RII)

#data_Noise["R_m_t"] = (data_Noise["RI"]+ data_Noise["RII"] - 200) / 2
data_Noise["R_m"] = (RI + RII) / 2
data_Noise["Z_m"] = (ZI + 400 - ZII) / 2

print(data_Noise)


   Visur_von Visur_nach        RI       RII        ZI       ZII       D  \
0          A          P  165.5625  365.5681  103.4893  296.5106     NaN   
1          A        PP1  365.7431  165.7520   96.7367  303.2727  27.423   
2        PP1          A  261.8117   61.8064  103.2586  296.7319  27.420   
3        PP1        PP2   60.9820  260.9672   97.2061  302.7818  25.437   
4        PP2        PP1   70.4547  270.4537  102.7893  297.2014  25.435   
5        PP2        PP3  251.6232   51.6080   98.3558  301.6405  33.468   
6        PP3        PP2  371.0540  171.0390  101.6236  298.3650  33.465   
7        PP3        PP4  151.6679  351.6638   99.3154  300.6764  23.316   
8        PP4        PP3  340.9883  140.9872  100.6859  299.3110  23.311   
9        PP4          E  168.9686  368.9751  100.4915  299.5136  44.822   
10         E        PP4  344.2161  144.2082   99.4958  300.5044  44.825   
11         E          Q  161.3028  361.3175  101.0000  299.0211     NaN   

        R_m_t        R_m

In [96]:
## Distances
def correction_met(temp, press, d): # watchout d in km!!! (pass as m)
    return (-1 * (temp - T) + 0.3 * (press - P)) * (d / 1000) # genäherte Korrektur

def correction_sehne(h_a, h_b, s):
    return np.sqrt( (s**2 - np.abs(h_a - h_b)) / 
                    ( (1 + (h_a / R)) * (1 + (h_b / R)) ) 
                    )

def calculate_h_b(h_a, d_s, zenith, i, t):
    return h_a + d_s * np.cos(radian(zenith)) + i - t

def correction_factor_distance_flat(X_m):
    return 1 + ((X_m**2) / (2*R**2))

def average(values):
    return sum(values) / len(values)

In [97]:
D_m = list()
D_m.append(pd.NA)
for i, dist in enumerate(data_Noise["D"]):
    if i == 0 or i == len(data_Noise["D"]) - 1: continue # ignore first and last entry
    if i % 2 != 0:
        D_m.append((dist + data_Noise["D"][i+1]) / 2)
        D_m.append((dist + data_Noise["D"][i+1]) / 2)
D_m.append(pd.NA)
data_Noise["D_m"] = D_m

In [ ]:
## Angles
def calc_brechungswinkel(ang_0, ang_1):
    return ang_1 - ang_0

def calculate_richtungswinkel(r_0):